<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/Linkdin_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Not relevant for now

In [ ]:
from flask import Flask, request, redirect
import requests

app = Flask(__name__)

# App credentials
CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"
REDIRECT_URI = "http://localhost:5000/callback"

# Route to start LinkedIn login
@app.route("/")
def home():
    return f'''
    <a href="https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}&scope=r_liteprofile%20r_emailaddress">
        Login with LinkedIn
    </a>
    '''

# Callback to handle LinkedIn response
@app.route("/callback")
def callback():
    code = request.args.get("code")
    token_url = "https://www.linkedin.com/oauth/v2/accessToken"
    data = {
        "grant_type": "authorization_code",
        "code": code,
        "redirect_uri": REDIRECT_URI,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }
    response = requests.post(token_url, data=data)
    access_token = response.json().get("access_token")

    # Fetch user profile
    headers = {"Authorization": f"Bearer {access_token}"}
    profile_response = requests.get("https://api.linkedin.com/v2/me", headers=headers)
    return profile_response.json()

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


**Job data scrape**

In [ ]:
!pip install playwright
!playwright install


In [ ]:
from playwright.async_api import async_playwright
import asyncio

async def scrape_job_posting(job_url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0")
        page = await context.new_page()

        await page.goto(job_url)
        await asyncio.sleep(5)  # Wait for 5 seconds

        # Saving the page's HTML content for debugging purposes
        html = await page.content()
        with open("page_debug.html", "w", encoding="utf-8") as f:
            f.write(html)
        print("Saved page content to page_debug.html")

        # Taking a screenshot of the page for verification
        await page.screenshot(path="screenshot.png", full_page=True)
        print("Screenshot saved as screenshot.png")


        # Extract job title
        job_title = await page.inner_text('h1.top-card-layout__title')  # Adjust selector
        company = await page.inner_text('a.topcard__org-name-link')  # Adjust selector
        location = await page.inner_text('span.topcard__flavor--bullet')  # Adjust selector
        description = await page.inner_text('div.show-more-less-html__markup')  # Adjust selector

        print(f"Job Title: {job_title}")
        print(f"Company: {company}")
        print(f"Location: {location}")
        print(f"Description: {description}")

        await browser.close()

job_url = "https://www.linkedin.com/jobs/view/4114601385/"
await scrape_job_posting(job_url)


Saved page content to page_debug.html
Screenshot saved as screenshot.png
Job Title: Computer Vision Algorithm Engineer
Company: Camtek 
Location:  Migdal HaEmek, North District, Israel
Description: Camtek designs, develops, manufactures and markets technologically advanced, cost-effective systems and related software products that are used to enhance processes and yields serving the Advanced Packaging, Memory, CMOS Image Sensors, MEMS, RF, and other segments in the Semiconductors industry.




Camtek invites you to work in a global, cutting-edge technology environment and be part of a market leader - join us and nurture your career.




Job description

Our Algorithm group is looking for a talented, motivated and ambitious Computer Vision Algorithm Engineer, to join us.

You will be working in an environment that offers endless learning and growth opportunities.




Responsibilities:

The position requires study and understanding of Camtek's existing algorithms, with the ability to mai

Start of profile scrape

In [ ]:
from playwright.async_api import async_playwright

async def scrape_profile(profile_url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(profile_url)2
        await asyncio.sleep(5)  # Wait for 5 seconds

        # Saving the page's HTML content for debugging purposes
        html = await page.content()
        with open("page_debug.html", "w", encoding="utf-8") as f:
            f.write(html)
        print("Saved page content to page_debug.html")

        # Taking a screenshot of the page for verification
        await page.screenshot(path="screenshot.png", full_page=True)
        print("Screenshot saved as screenshot.png")

        name = await page.inner_text('css-selector-for-name')
        title = await page.inner_text('css-selector-for-title')
        await browser.close()
        return {"name": name, "title": title}

profile_url = "https://www.linkedin.com/in/daniel-behar-168647280/"
profile_data = await scrape_profile(profile_url)
print(profile_data)


Saved page content to page_debug.html
Screenshot saved as screenshot.png


TimeoutError: Page.inner_text: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("css-selector-for-name")
